In [ ]:
import sys
import pathlib
src_path = pathlib.Path().absolute().parent /"src"
sys.path.append(str(src_path))
import data_functions
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
import pickle
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train = data_functions.get_dataframes()

In [ ]:
x_train.drop(['date_recorded','installer','funder','wpt_name', 'subvillage','ward','recorded_by','scheme_name','scheme_management','extraction_type',
             'extraction_type_class','payment','public_meeting','permit','management','management_group','source','source_class',
             'waterpoint_type_group','latitude','longitude','num_private','region_code','district_code'], inplace=True, axis=1)

x_test.drop(['date_recorded','installer','funder','wpt_name', 'subvillage','ward','recorded_by','scheme_name','scheme_management','extraction_type',
             'extraction_type_class','payment','public_meeting','permit','management','management_group','source','source_class',
             'waterpoint_type_group','latitude','longitude','num_private','region_code','district_code'], inplace=True, axis=1)

In [ ]:
x_train_nums= x_train.select_dtypes(exclude="object")
x_train_cat= x_train.select_dtypes(include="object")

In [ ]:
ohe=OneHotEncoder(drop='first', sparse=False)

In [ ]:
x_train_ohe=pd.DataFrame(ohe.fit_transform(x_train_cat), columns= ohe.get_feature_names(x_train_cat.columns), index= x_train_cat.index)
sum(x_train_ohe.isna().sum())

In [ ]:
x_train_ohe.shape

In [ ]:
si=SimpleImputer()
x_nums_si=pd.DataFrame(si.fit_transform(x_train_nums), index= x_train_nums.index, columns= x_train_nums.columns)

In [ ]:
scale= StandardScaler()
x_train_nums_scaled= pd.DataFrame(scale.fit_transform(x_nums_si), index= x_nums_si.index, columns= x_nums_si.columns)

In [ ]:
x_final= x_train_nums_scaled.join(x_train_ohe)

In [ ]:
x_final.head()

In [ ]:
x_val, x_val_test, y_val, y_val_test= train_test_split(x_final, y_train, random_state=2020)

In [ ]:
from imblearn.over_sampling import SMOTE
sm1 = SMOTE(random_state=2020)

In [ ]:
X_tr_sm, y_tr_sm = sm1.fit_resample(x_val, y_val)

In [ ]:
X_tr_sm.shape

In [ ]:
rf=RandomForestClassifier()
rf.fit(X_tr_sm,y_tr_sm)
y_pred_rf_1=rf.predict(x_val_test)

In [ ]:
print(classification_report(y_val_test, y_pred_rf_1))

In [ ]:
from sklearn.metrics import plot_confusion_matrix

In [ ]:
plot_confusion_matrix(rf, x_val_test,y_val_test)

In [ ]:
params = {'max_depth': [5,10,25],
    'min_samples_split': [3,5,7],
    'max_features': ['sqrt', 0.5],
    'n_estimators': [50, 100, 200]}

In [ ]:
rf_1_grid_search=GridSearchCV(rf,params,cv=3, return_train_score= True) #feel free to remove return_train_score


In [ ]:
rf_1_grid_search.fit(X_tr_sm, y_tr_sm)